💌**假设我们想对函数$y=2\textbf{x}^T\textbf{x}$关于列向量$\textbf{x}$求导**

In [1]:
import torch

x=torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

🥗**在计算y关于$\textbf{x}$的梯度之前，需要一个地方存储梯度**

In [2]:
x.requires_grad_(True)
# 这里就是表明 需要把x的梯度存下来
# 注意这里是requires_grad_，最后有个横杠，这个是函数
# 不是requires_grad ，这个是属性/实例
# 等价于 x=torch.arange(4.0，requires_grad=True)
x.grad 
# 默认是None，还没有计算过

然后计算y

In [4]:
y=2*torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

🍤**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**

In [5]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

这里可以自己算一下，$y=2x^2$，y对x的导数就是$4x$，看看x.grad和x向量的值

In [7]:
x.grad/x

tensor([nan, 4., 4., 4.], grad_fn=<DivBackward0>)

In [9]:
x.grad==4*x

tensor([True, True, True, True])

🍔**现在计算x另一个函数，求另一波导数**

In [14]:
# 默认情况下，pytorch会累计梯度，
# 所以针对同一个变量求导时，需要清楚之前的值

x.grad.zero_() # 把x的梯度信息清零

y=x.sum()
# 新的函数关系，这个条件下，求y对x的导数
y.backward()
x.grad
# x.grad返回的是代入x值之后的导数，
# 这里x.sum() y对x的导数就是常量，相当于dy/dx=x/x=1

# 可以看一下注释掉x.grad.zero_()这行之后是什么样子，梯度会累计相加

tensor([1., 1., 1., 1.])

🤡**深度学习中，目的不是计算微分矩阵，而是`批量中`每个样本`单独计算`的`偏导数之和`**

In [15]:
# 对非标量调用`backward`需要传入一个`gradient`参数，
# 该参数指定微分函数
x.grad.zero_()
# 老规矩，先清零

y=x*x


y.sum().backward()
# 等价于y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

+ 前面两个例子y都是标量，一个数值
+ 这里给的y是向量 所以理论上y对x的梯度是个矩阵
+ 但是深度学习很少做这样的事，很少对一个向量的函数求导，所以大部分情况下，会对y做一个求和，把y变成标量，然后对标量进行求导
+ 不是所有情况都会求和，但是要想办法把它变成一个还可以的标量

In [16]:
x.grad==2*x

tensor([True, True, True, True])

$y=x^2$这里求y对x的导数，结果就是$\frac{dy}{dx}=2$

🎂**将某些计算移动到记录的计算图之外**

In [17]:
x.grad.zero_()

y=x*x
u=y.detach()

# Returns a new Tensor, detached from the current graph.
# The result will never require gradient.
z=u*x

z.sum().backward()
x.grad==u

tensor([True, True, True, True])

In [20]:
x,u,y

(tensor([0., 1., 2., 3.], requires_grad=True),
 tensor([0., 1., 4., 9.]),
 tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>))

+ detach函数会返回一个新张量，这个张量从当前计算图中分离，
+ 同时这个结果不会要求梯度计算
+ 对y进行detach（），赋值给u，那么u就是y去除是x函数这一属性之后的结果，或者可以理解为y不再是x的函数，而是一个常量，u就不是x的函数了
+ 可以看到打印结果，x是需要计算梯度的，u是没有梯度的，所以不再有函数表达，直接代入值，只有计算结果
+ 而虽然是对y进行detach操作，但是并没有对y造成影响，依然是梯度函数，grad_fn，只是对这个操作生成的新的张量有影响
+ $z=u*x$，所以z关于x的导数$\frac{dux}{dx}=u$

In [21]:
x.grad.zero_()
y.sum().backward()
x.grad==2*x

tensor([True, True, True, True])

虽然u没有梯度，但是并不对y有影响，y依然是x的函数，可以计算梯度

🍱**即使构建函数的计算图需要通过python控制流（例如：条件、循环或任意函数调用），仍然可以计算得到变量的梯度**

In [26]:
def f(a):
    b=a**2
    while b.norm()<1000:
        b=b*2
    if b.sum()>0:
        c=b
    else:
        c=100*b
    return c

a=torch.randn(size=(),requires_grad=True)
# size等于空，也就是生成一个常量
d=f(a)
# d对a求导，函数就是f，但是这个函数关系中含有控制流，
d.backward()
# 但是这里依然可以求导

`b.norm()`这个是求范数，
+ 如果b是向量，则就是L2范数，平方求和再开根号
+ 如果b是矩阵，则就是F范数，也是平方求和再开根号

这里使用隐式构造而不是显式构造导数，这样可以计算复杂的流，但是计算会比显式构造要慢一些

+ 显式构造是先把整个计算构造出来，再去给值，简单来说，就是用python实现一个函数和使用数学实现一个函数是不一样的

In [28]:
a,a.grad

(tensor(1.8705, requires_grad=True), tensor(1915.3682))

In [33]:
# 可以逐步验证一下控制流的走向
b=a**2
print(b)
# 这里b设置了一个while循环，要一直b对自己求平方，直到>1000才可以继续
print(b.norm())
# 标量的norm等于本身的绝对值
print(b.sum())

tensor(3.4987, grad_fn=<PowBackward0>)
tensor(3.4987, grad_fn=<CopyBackwards>)
tensor(3.4987, grad_fn=<SumBackward0>)


In [30]:
c=torch.rand(size=(1,1))
c
# 关于size参数， 
"""
size (int...): a sequence of integers defining the shape of the output tensor.
Can be a variable number of arguments or a collection like a list or tuple.

一个用于定义输出向量形状的整数序列
可以是一个值可变的变量，或者是类似list或者tuple的集合
"""

tensor([[0.0096]])